In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/AmazonMLChallenge/dataset
!pip install transformers

best_model_state.bin   sdd.csv		      train.csv
best_model_state.pt    test.csv		      train_preprocessed.csv
sample_submission.csv  test_preprocessed.csv
     |████████████████████████████████| 2.6 MB 7.9 MB/s 
     |████████████████████████████████| 895 kB 56.0 MB/s 
     |████████████████████████████████| 636 kB 58.2 MB/s 
     |████████████████████████████████| 3.3 MB 60.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
import numpy as np
import csv
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import gc
gc.collect()
torch.cuda.empty_cache()
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import regex as re
from sklearn import preprocessing
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device 

device(type='cuda', index=0)

In [4]:
TRAIN_DATA_PATH = '/content/drive/MyDrive/AmazonMLChallenge/dataset/train.csv'
TEST_DATA_PATH = '/content/drive/MyDrive/AmazonMLChallenge/dataset/test.csv'
train_data = pd.read_csv(TRAIN_DATA_PATH, escapechar='\\', quoting=csv.QUOTE_NONE)
train_data = train_data.sample(n=100000)
train_data.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
2297143,"Amsik®Blind Spot Round Mirror Black, (Real Gla...",The mirrors easily mount to the vehicle's left...,"[It is small, but it is smart mirror, and work...",AMSIK,32296
2078681,Nivea Face Sun Block Whitening Cream SPF 50 Pa...,Nivea Sun Fac Sun Block Whitening Cream Spf 50...,NIVEA face SUN Block Whitening Cream SPF 50 PA ++,Nivea,35
1125035,ROZVEH Women's Shift Brown Dress,"<p>Super Fine Rayon,&nbsp;Brown Floral Print d...","[Material: Super Fine Rayon, Brown Floral Prin...",ROZVEH,1724
1928801,Narayan Creation Pashmina Palazzo Suit Dupatta...,Narayan creatins pashmina unstitched palazzo s...,NaN,Generic,465
864509,Q-rious Women's Silicon Strap Spagettis/Camiso...,Trendy Q-rious Spagetti with sequin detachable...,"[Silicon and Elastic strap option,Detachable S...",Q-rious,2003


In [5]:
test_data = pd.read_csv(TEST_DATA_PATH, escapechar='\\', quoting=csv.QUOTE_NONE)
test_data.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [6]:
# print('Sample TITLE:\n', train_data.loc[0, 'TITLE'])
# print('\nSample DESCRIPTION:\n', train_data.loc[0, 'DESCRIPTION'])
# print('\nSample BULLET_POINTS:\n', train_data.loc[0, 'BULLET_POINTS'])

In [7]:
print('No of unique TITLEs:', len(train_data.TITLE.unique()))
print('No of unique BRANDs:', len(train_data.BRAND.unique()))

No of unique TITLEs: 99448
No of unique BRANDs: 36649


In [8]:
def preprocess(x):
    x = x.replace(',', ' ')
    x = x.replace(';', ' ')
    x = x.replace(':', ' ')
    x = x.replace('-', ' ')
    x = x.replace('[', ' ')
    x = x.replace(']', ' ')
    x = x.replace('(', ' ')
    x = x.replace(')', ' ')
    x = x.replace('{', ' ')
    x = x.replace('}', ' ')

    # To convert cases like 14. 5 or 14 .5 or 14 . 5 or 14.5 to 145,
    # since tokernizer won't handle decimal numbers
    x = re.sub(r"(\d)\s*\.\s*(\d)", r"\1\2", x)
    x = re.sub(r"\b\d+\b", ' ', x)

    x = ' '.join(x.split())

    return x

def combine_attributes(row):
    return row.TITLE + ' ' + row.BULLET_POINTS


In [9]:
# Preprocessing the textual data
# 1. Removing commas, brackets, colons, semicolons
# 2. Removing excess whitespaces within lines
# 3. Joining bullet points into a single line
# 4. Replacing NaN bullet points by description if it's not NaN, otherwise replace by a "nil"
#
# Joining data as < [CLS] title [SEP] bullet_points > for feeding into the BERT model

# for data in [train_data, test_data]:
#     data['TITLE'] = data.TITLE.apply(preprocess)
#     data['DESCRIPTION'] = data.DESCRIPTION.apply(preprocess)
#     data['BULLET_POINTS'] = data.BULLET_POINTS.apply(preprocess)
#     data['TEXT'] = data.apply(combine_attributes, axis=1)

# train_data.head()

In [10]:
# train_data.to_csv('/content/drive/MyDrive/AmazonMLChallenge/dataset/train_preprocessed.csv')
# test_data.to_csv('/content/drive/MyDrive/AmazonMLChallenge/dataset/test_preprocessed.csv')

In [11]:
!ls /content/drive/MyDrive/AmazonMLChallenge/dataset/

best_model_state.bin   sdd.csv		      train.csv
best_model_state.pt    test.csv		      train_preprocessed.csv
sample_submission.csv  test_preprocessed.csv


In [12]:
 PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
 tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
 sample_txt = 'Best place that I have visited? Iceland was the most beautiful and I consider myself lucky to have visited Iceland at such an early age.'
 #sample_txt = 'When was I last outside? I am stuck at home for 2 weeks.'
 tokens = tokenizer.tokenize(sample_txt)
 token_ids = tokenizer.convert_tokens_to_ids(tokens)
 print(f'  Sentence: {sample_txt}')
 print(f'\n  Tokens: {tokens}')
 print(f'\n  Token IDs: {token_ids}')   # Each token has a unique ID for the model to understand what we are referring to.
 len(tokens)


  Sentence: Best place that I have visited? Iceland was the most beautiful and I consider myself lucky to have visited Iceland at such an early age.

  Tokens: ['best', 'place', 'that', 'i', 'have', 'visited', '?', 'iceland', 'was', 'the', 'most', 'beautiful', 'and', 'i', 'consider', 'myself', 'lucky', 'to', 'have', 'visited', 'iceland', 'at', 'such', 'an', 'early', 'age', '.']

  Token IDs: [2190, 2173, 2008, 1045, 2031, 4716, 1029, 10399, 2001, 1996, 2087, 3376, 1998, 1045, 5136, 2870, 5341, 2000, 2031, 4716, 10399, 2012, 2107, 2019, 2220, 2287, 1012]


27

In [13]:
## mapping from browse_id to 0...9918
lb = preprocessing.LabelBinarizer()
l = lb.fit(list(train_data.BROWSE_NODE_ID))
og_to_mapped = {}
mapped_to_og = l.classes_
for i in range(len(mapped_to_og)):
    og_to_mapped[mapped_to_og[i]] = i

In [14]:
## mapping brand names to integer labels
le = preprocessing.LabelEncoder()
l = le.fit(list(train_data.BRAND))
brand_to_mapped = {}
mapped_to_brand = l.classes_
for i in range(len(mapped_to_brand)):
    brand_to_mapped[mapped_to_brand[i]] = i

In [15]:
class AmazonDataset(Dataset):

  def __init__(self, df, tokenizer, max_len):
    self.df = df                   # Full df with columns TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
    self.tokenizer = tokenizer     # Tokenizer is the BERT_Tokenizer.
    self.max_len = max_len         # max_length of each sequence.
  
  def __len__(self):
    return len(self.df)            # Size of dataset
  
  def __getitem__(self, item):
    bullet_pts_str = str(self.df.BULLET_POINTS[item])
    bullet_pts_str = ". ".join(bullet_pts_str[1:-1].split(",")) if bullet_pts_str[0]=='['  else bullet_pts_str
    title_str = str(self.df.TITLE[item])
    input = preprocess(title_str) + " [SEP] " + preprocess(bullet_pts_str)      # returns the input at the row == item
    input_brand = torch.as_tensor(brand_to_mapped.get(str(self.df.BRAND[item]), 36181), dtype=torch.int32)    # returns the brand input at the row == item
    target = og_to_mapped[int(self.df.BROWSE_NODE_ID[item])]                    # returns the string of targets at the row == item

    encoding = self.tokenizer.encode_plus(
      input,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      truncation='only_first',
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'input_text': input,
      'input_brand': input_brand,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)            # dictionary containing all the features is returned.
    }

In [16]:
df_train, df_val = train_test_split(train_data, test_size=0.3)
df_train = df_train.reset_index()
df_val = df_val.reset_index()

In [17]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = AmazonDataset(
    df=df,
    tokenizer=tokenizer,
    max_len=max_len
  )                                  # Dataset would be created which can be used to create and return dataloader.

  return DataLoader(
    ds,
    batch_size=batch_size,
    #num_workers=4
  )

BATCH_SIZE = 56
MAX_LEN = 128
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
#test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)
print(type(next(iter(train_data_loader))))
data = next(iter(train_data_loader))
data.keys()
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)
print(data['targets']) 

<class 'dict'>
torch.Size([56, 128])
torch.Size([56, 128])
torch.Size([56])
tensor([ 678,  541,  891, 1611,  613,  970, 1438,   62,  530,   63,  622, 1199,
         596,  491,   85,  689, 1572, 1190,  610,  724,  710, 2164,  701, 2098,
         623,  432,  680,  609,  178, 1958,  941,  484,  607,  572, 2566, 2833,
         673,  877,  811,  301,  249,  622,  762,  623,  828,  682,  623,  215,
        2551,  818,  468,  627, 2798,  827,  697, 1469])


In [18]:
 bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
 """
 model_test = bert_model(
   input_ids=encoding_test['input_ids'], 
   attention_mask=encoding_test['attention_mask']
 )
 model_test.keys()
 last_hidden_state=model_test['last_hidden_state']
 pooled_output=model_test['pooler_output']
 last_hidden_state.shape
 bert_model.config.hidden_size
 pooled_output.shape
 """
 class ProductClassifier(nn.Module):
   def __init__(self, n_classes):
     super(ProductClassifier, self).__init__()
     self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
     self.drop = nn.Dropout(p=0.3)                                 ## For regularization with dropout probability 0.3.
     self.out = nn.Linear(self.bert.config.hidden_size+1, n_classes) ## append an Output fully connected layer representing the number of classes
     self.brandL1 = nn.Linear(1,1)
     self.brandAct = nn.ReLU()
     
   def forward(self, brand_id, input_ids, attention_mask):
     returned = self.bert(
       input_ids=input_ids,
       attention_mask=attention_mask
     )
     brand_out = self.brandAct(self.brandL1(brand_id.float()))
     pooled_output = returned["pooler_output"]
     output = self.drop(pooled_output)
     cat_output = torch.cat((brand_out, output), dim=1)
     return self.out(cat_output)
 
 model = ProductClassifier(9919)
 model = model.to(device)
 input_ids = data['input_ids'].to(device)
 attention_mask = data['attention_mask'].to(device)
 brand_id = data['input_brand'].reshape((-1, 1)).to(device)
 print(input_ids.dtype)
 print(input_ids.shape)      # batch size x seq length
 print(attention_mask.shape) # batch size x seq length
 print(brand_id.shape)       # batch size x seq length
 F.softmax(model(brand_id, input_ids, attention_mask), dim=1)
 EPOCHS = 10
 optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
 total_steps = len(train_data_loader) * EPOCHS    # Number of batches * Epochs (Required for the scheduler.)
 scheduler = get_linear_schedule_with_warmup(
   optimizer,
   num_warmup_steps=0,     # Recommended in the BERT paper.
   num_training_steps=total_steps  
 )
 loss_fn = nn.CrossEntropyLoss().to(device) 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

torch.int64
torch.Size([56, 128])
torch.Size([56, 128])
torch.Size([56, 1])


In [19]:
print(bert_model.config.hidden_size)

768


In [20]:
 def train_epoch(
   model, 
   data_loader, 
   loss_fn, 
   optimizer, 
   device, 
   scheduler, 
   n_examples
 ):
   model = model.train()    # To make sure that the droupout and normalization is enabled during the training.
   losses = []
   correct_predictions = 0
   for d in data_loader:
     input_ids = d["input_ids"].to(device)
     attention_mask = d["attention_mask"].to(device)
     brand_id = d["input_brand"].reshape((-1, 1)).to(device)
     targets = d["targets"].to(device)
     outputs = model(
       brand_id=brand_id,
       input_ids=input_ids,
       attention_mask=attention_mask
     )
     max_prob, preds = torch.max(outputs, dim=1)    # Returns 2 tensors, one with max_probability and another with the respective predicted label.
     loss = loss_fn(outputs, targets)
     correct_predictions += torch.sum(preds == targets)
     losses.append(loss.item())
     loss.backward()     # Back_Propogation
     nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Recommended by the BERT paper to clip the gradients to avoid exploding gradients.
     optimizer.step()
     scheduler.step()
     optimizer.zero_grad()
   return correct_predictions.double() / n_examples, np.mean(losses)     # Return the mean loss and the ratio of correct predictions. 

In [21]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()        # To make sure that the droupout and normalization is disabled during the training.

  losses = []
  correct_predictions = 0

  with torch.no_grad():        # Back propogation is not required. Torch would perform faster. 
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      brand_id = d["input_brand"].reshape((-1, 1)).to(device)
      targets = d["targets"].to(device)

      outputs = model(
        brand_id = brand_id,
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      max_prob, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [22]:
# checkpoint = torch.load('/content/drive/MyDrive/AmazonMLChallenge/abismodel/best_model_state.pt')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [23]:
# del checkpoint

In [ ]:
%%time

history = defaultdict(list)          # Similar to Keras library saves history
best_accuracy = 0                    

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch+1
    }, '/content/drive/MyDrive/AmazonMLChallenge/abismodel/best_model_state.pt')
    best_accuracy = val_acc

Epoch 1/20
----------
Train loss 4.96865697593689 accuracy 0.41559999999999997
Val   loss 3.7783376169738485 accuracy 0.5337000000000001

Epoch 2/20
----------
Train loss 3.433847014904022 accuracy 0.5605


In [27]:
!mkdir -p /content/drive/MyDrive/AmazonMLChallenge/abismodel/

In [28]:
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
#     'epoch': 4
# }, '/content/drive/MyDrive/AmazonMLChallenge/abismodel/best_model_state.pt')

In [29]:
class AmazonTestDataset(Dataset):

  def __init__(self, df, tokenizer, max_len):
    self.df = df                   # Full df with columns TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
    self.tokenizer = tokenizer     # Tokenizer is the BERT_Tokenizer.
    self.max_len = max_len         # max_length of each sequence.
  
  def __len__(self):
    return len(self.df)            # Size of dataset
  
  def __getitem__(self, item):
    bullet_pts_str = str(self.df.BULLET_POINTS[item])
    bullet_pts_str = ". ".join(bullet_pts_str[1:-1].split(",")) if bullet_pts_str[0]=='['  else bullet_pts_str
    title_str = str(self.df.TITLE[item])
    input = preprocess(title_str) + " [SEP] " + preprocess(bullet_pts_str)    # returns the input at the row == item
    input_brand = torch.as_tensor(brand_to_mapped.get(str(self.df.BRAND[item]), 36181), dtype=torch.int32)       # returns the brand input at the row == item
    # 36181 - brand_to_mapped['nan']
    
    encoding = self.tokenizer.encode_plus(
      input,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      truncation='only_first',
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'input_text': input,
      'input_brand': input_brand,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      #'targets': torch.tensor(target, dtype=torch.long)            # dictionary containing all the features is returned.
    }

In [58]:
print(brand_to_mapped['nan'])

36181


In [30]:
def create_test_data_loader(df, tokenizer, max_len, batch_size):
  ds = AmazonTestDataset(
    df=df,
    tokenizer=tokenizer,
    max_len=max_len
  )                                  # Dataset would be created which can be used to create and return dataloader.

  return DataLoader(
    ds,
    batch_size=batch_size,
    #num_workers=4
  )

test_data_loader = create_test_data_loader(test_data, tokenizer, MAX_LEN, BATCH_SIZE)
test_output = []
with torch.no_grad():        # Back propogation is not required. Torch would perform faster. 
  for d in test_data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    brand_id = d["input_brand"].reshape((-1, 1)).to(device)
    outputs = model(
      brand_id=brand_id,
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    max_prob, preds = torch.max(outputs, dim=1)
    preds = preds.cpu()
    for pred in preds:
      test_output.append(mapped_to_og[pred])




In [31]:
test_data['BROWSE_NODE_ID'] = test_output
test_out = test_data.drop(['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND'], axis=1)
test_out.to_csv("/content/drive/MyDrive/AmazonMLChallenge/abismodel/sdd.csv",index = False)